## Find all positive and negative features of each pizza shop
1. Tokenize each review into sentences
2. Classify the senitment of each sentence, place a prior with the review score
3. Tag each noun in each sentence
4. Is that noun used in a positive sentence or negative sentence?

In [1]:
from app import app,db

import pandas as pd
import numpy as np
import json
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

import nltk
from nltk.tag import StanfordNERTagger
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.corpus import stopwords
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

/Users/mayen/anaconda/envs/python35/lib/python3.5/site-packages/flask_sqlalchemy/__init__.py:800: UserWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.
  warnings.warn('SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.')


Creating DB


In [2]:
reviews = pd.read_sql_table('reviews',app.config['SQLALCHEMY_DATABASE_URI'])

In [3]:
business = pd.read_sql_table('business',app.config['SQLALCHEMY_DATABASE_URI'])
category = pd.read_sql_table('category',app.config['SQLALCHEMY_DATABASE_URI'])

In [5]:
pizza = business.merge(category[category['category']=='Pizza'],
                       how='inner',
                       on='business_id',
                      copy=False)
pizza.drop(['id_x','id_y'],axis=1,inplace=True)
pizza.head()

,business_id,full_address,open,city,state,name,stars,review_count,lon,lat,price_range,type,category
0,6o3RK6rTcN3nw-j-r2nQmA,"231 E Main St\nCarnegie\nCarnegie, PA 15106",True,Carnegie,PA,Porto Fino Pizzaria & Gyro,2.5,4,-80.084934,40.408832,NaN,business,Pizza
1,t_gan0EXAw8csKIeFyazJw,"820 Amity St\nHomestead\nHomestead, PA 15120",True,Homestead,PA,Homestead Capri Pizza,2.0,4,-79.910527,40.406705,1.0,business,Pizza
2,-taSf0shFSUKy9iCXnySsA,"2615 Main St\nHomestead, PA 15120",True,Homestead,PA,Italian Village Pizza,2.5,6,-79.900010,40.392552,NaN,business,Pizza
3,6p9TlP2I8S4agxYMNFSDjg,205 East Waterfront Drive\nHomestead\nHomestea...,True,Homestead,PA,Uno Pizzeria & Grill,3.0,55,-79.914136,40.410967,2.0,business,Pizza
4,uLnDFdn011hEroQQMLmxNQ,"3502 Main St\nMunhall, PA 15120",True,Munhall,PA,The Pizza Company,4.0,12,-79.903300,40.385718,1.0,business,Pizza


In [6]:
pizza_reviews = reviews.merge(category[category['category']=='Pizza'],
                              how='inner',
                              on='business_id',
                             copy=False)
pizza_reviews.drop(['id_x','id_y'],axis=1,inplace=True)

In [13]:
review = pizza_reviews.sample(1)
rev = review['text'].values[0]
blob = TextBlob(rev)

In [16]:
blob.sentiment
review['stars']

112491    5
Name: stars, dtype: int64

In [54]:
def stars_prior(score):
    """convert the review score (stars)
    to a sentiment score between -1 and 1
    min(score) = 1
    max(score) = 5
    """
    assert score <= 5
    assert score >= 1
    return (score-3)/2
    
def extract_nouns(row,alpha=0.1):
    """Input is a row of the review dataframe"""
    text = row['text']
    sentiment = stars_prior(row['stars'])
    threshold = -alpha*sentiment
    print(sentiment,threshold)
    #TODO use review sentiment as prior
    # for sentence sentiment
    blob = TextBlob(text)
    words = {'positive':set(),'negative':set()}
    for sentence in blob.sentences:
        sentiment = ['negative','positive'][int(sentence.polarity>threshold)]
        words[sentiment] = words[sentiment].union(set(sentence.noun_phrases))
    return words

In [55]:
print(review['stars'])
extract_nouns(review.iloc[0])

112491    5
Name: stars, dtype: int64
1.0 -0.1


{'negative': set(),
 'positive': {'cesar',
  'cheese sauce',
  'chicken wrap',
  'delivery',
  'discontinue business',
  'french fries',
  'greasey',
  'love',
  'pittsburgh',
  'pizza joint',
  'real cheese'}}

In [38]:
blobs = TextBlob(rev.values[0]).sentences
b = blobs[3]
print(b)

The pizzas were okay...definitely not spectacular like many others are saying, but not bad...NO WAY is this the BEST in the US!


Sentiment(polarity=0.39999999999999997, subjectivity=0.5666666666666667)